In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# モデル定義
def create_model():
    model = Sequential()
    model.add(Dense(1024,activation="relu", input_shape=(14,256))) # tf.keras.layers.Conv1D 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,activation="relu")) 
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 損失関数はbinary_crossentropy
    return model

2023-05-24 18:15:15.657142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# データの読み込み
df = pd.read_csv('Raw.csv')

# 1秒ごとのデータに分割
n_samples_per_second = 256  # 256Hzのサンプリングレート
total_seconds = 10  # 全体の秒数

# ラベルの作成（5秒閉じ、5秒開け）
labels_close = [0] * 5 
labels_open = [1] * 5 
labels = labels_close + labels_open
labels = to_categorical(labels)

# データとラベルを1秒ごとに分割
X = np.array([df.iloc[i*n_samples_per_second:(i+1)*n_samples_per_second, :-1].values for i in range(total_seconds)])

X = X.transpose(2,0,1)
for i in range(14):
    scaler = StandardScaler()
    X[i] = scaler.fit_transform(X[i])
X = X.transpose(1,0,2)

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

# モデル作成
model = create_model()

# 学習
model.fit(X_train, y_train, epochs=10, batch_size=2, verbose=1)

# 訓練データに対する評価
train_scores = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Accuracy: {train_scores[1]*100}")

# テストデータに対する評価
test_scores = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_scores[1]*100}")


Epoch 1/10
4/4 [==============================] - 1s 3ms/step - loss: 1.0355 - accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 0s 2ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 0s 2ms/step - loss: 4.2991e-05 - accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 2ms/step - loss: 9.1638e-06 - accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 2ms/step - loss: 1.8803e-06 - accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 2ms/step - loss: 2.7761e-07 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 2ms/step - loss: 1.3592e-07 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 0s 2ms/step - loss: 6.2197e-08 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 0s 2ms/step - loss: 2.9687e-08 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 87ms/step - loss: 1.6796e-08 - accuracy: 1.0000
Traini

In [3]:
np.array(X).shape

(10, 14, 256)

In [4]:
y_train.shape

(8, 2)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14, 1024)          263168    
                                                                 
 flatten (Flatten)           (None, 14336)             0         
                                                                 
 dense_1 (Dense)             (None, 512)               7340544   
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 7,604,738
Trainable params: 7,604,738
Non-trainable params: 0
_________________________________________________________________
